In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
from sklearn import model_selection
from PIL import Image
import time
import pandas as pd

import os
from scipy import ndimage, misc
import glob
import pickle
import multiprocessing as mp

/Users/avinashk94/anaconda3/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
data = pd.read_csv('files/Anno/list_attr_celeba.txt', delim_whitespace = True, header=1)
df = data['Eyeglasses']

In [3]:
df.shape

(202599,)

In [4]:
usingImages = 4000
nClasses= 2

In [5]:
images = np.array([np.array(Image.open("files/Img/img_align_celeba/"+str(fname))) for fname in df.head(usingImages).index])
images = np.float32(images)/256
with open('my.pickle', 'wb') as handle:
    pickle.dump(images, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [6]:
with open('my.pickle', 'rb') as handle:
    images = pickle.load(handle)

In [ ]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(images, df.head(usingImages), test_size=0.20, shuffle=False)
X_validate, X_test, y_validate, y_test = model_selection.train_test_split(X_test, y_test, test_size=0.50, shuffle=False)

In [ ]:
X_train.shape

In [ ]:
df.head(usingImages).shape

In [ ]:
from matplotlib.pyplot import imshow
%matplotlib inline
imshow(X_train[0])

In [ ]:
x = tf.placeholder(tf.float32, [None, 218, 178, 3])
t = tf.placeholder(tf.float32, [None,2])

In [ ]:
def convolve2D(x,W):
    return tf.nn.conv2d(x,W,strides=[1,1,1,1], padding='SAME')

In [ ]:
def maxpool2d(x):
    return tf.nn.max_pool(x,ksize=[1,2,2,1] ,strides=[1,2,2,1], padding='SAME')

In [30]:
def convolutionalNeuralNetwork(x):
    weights = {'conv1':tf.Variable(tf.random_normal([5,5,3,64])),
               'conv2':tf.Variable(tf.random_normal([5,5,64,64])),
               'fullyC1':tf.Variable(tf.random_normal([7*7*64,1024])),
               'fullyC2':tf.Variable(tf.random_normal([1024,1024])),
               'out':tf.Variable(tf.random_normal([1024,nClasses]))}
    
    biases = {'conv1':tf.Variable(tf.random_normal([64])),
              'conv2':tf.Variable(tf.random_normal([64])),
              'fullyC1':tf.Variable(tf.random_normal([1024])),
              'fullyC2':tf.Variable(tf.random_normal([1024])),
              'out':tf.Variable(tf.random_normal([nClasses]))}
    
#     x = tf.transpose(x)
    
#     Img = tf.reshape(x,[-1,28,28,1])
    
    conv1 = tf.nn.relu(convolve2D(x,weights['conv1']) + biases['conv1'])
    conv1 = maxpool2d(conv1)
    print(conv1)
    
    conv2 = tf.nn.relu(convolve2D(conv1,weights['conv2']) + biases['conv2'])
#     print(conv2)
    conv2 = maxpool2d(conv2)
    print(conv2)
    
    conv2 = tf.reshape(conv2,[-1,7*7*64])
#     print(conv2)
    fcLayer1 = tf.nn.relu(tf.matmul(conv2,weights['fullyC1']) + biases['fullyC1'])
#     print(fcLayer1)
    fcLayer1 = tf.nn.dropout(fcLayer1,keepRate)
    fcLayer2 = tf.nn.relu(tf.matmul(fcLayer1,weights['fullyC2']) + biases['fullyC2'])
    output = tf.matmul(fcLayer2,weights['out']) + biases['out']
    print(output)
    
    regLoss = tf.nn.l2_loss(weights['conv1']) + tf.nn.l2_loss(weights['conv2']) + tf.nn.l2_loss(weights['fullyC1']) + tf.nn.l2_loss(weights['fullyC1']) + tf.nn.l2_loss(weights['out'])
    return tf.transpose(output), regLoss

In [31]:
def trainNetwork():
    with tf.name_scope("loss") as scope:
        loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=tf.transpose(y),labels=tf.transpose(t)) + lambdaa*regLoss)
        tf.summary.scalar("loss",loss)
    
    with tf.name_scope("training") as scope:
        optimizer = tf.train.AdamOptimizer(learning_rate=learningRate).minimize(loss)
    
    with tf.name_scope("accuracy") as scope:
        correct = tf.equal(tf.argmax(y),tf.argmax(t))
        accuracy = tf.reduce_mean(tf.cast(correct,'float'))
        
    init = tf.global_variables_initializer()
    sess = tf.Session()
    mergeSummary = tf.summary.merge_all()

    sess.run(init)
    summaryWriter = tf.summary.FileWriter('../../TFout/2', sess.graph)

    for epoch in range(nEpochs):
        error = 0.0
        for i in range(int(m/batchSize)):
            batch_xs, batch_ys = mnist.train.next_batch(batchSize)
            _, er, summaryStr = sess.run([optimizer,loss,mergeSummary],feed_dict={x:batch_xs.T,t:batch_ys.T})
            summaryWriter.add_summary(summaryStr, epoch*(int(m/batchSize)) + i)
            error += er
        if (epoch+1)%printEvery == 0:
            print('Loss in ',epoch+1,' epoch is ',error)

    prediction = tf.equal(tf.argmax(y),tf.argmax(t))
    accuracy = tf.reduce_mean(tf.cast(prediction,"float"))
    print("Accuracy Train:", sess.run(accuracy,{x: trainImages, t: trainLabels}))
    print("Accuracy validation:", sess.run(accuracy,{x: validationImages, t: validationLabels}))
    print("Accuracy Test:", sess.run(accuracy,{x: testImages, t: testLabels}))
    print("USPS Test Accuracy:", sess.run(accuracy,{x: uspsImages, t: uspsLabels}))

In [32]:
nEpochs = 2
keepRate = 0.8
start_time = time.time()
y, regLoss = convolutionalNeuralNetwork(X_train)
trainNetwork()
print("--- %s seconds ---" % (time.time() - start_time))

Tensor("MaxPool:0", shape=(800, 109, 89, 64), dtype=float32)
Tensor("MaxPool_1:0", shape=(800, 55, 45, 64), dtype=float32)


ValueError: Dimension size must be evenly divisible by 3136 but is 126720000 for 'Reshape' (op: 'Reshape') with input shapes: [800,55,45,64], [2] and with input tensors computed as partial shapes: input[1] = [?,3136].